In [ ]:
# ==========================================
# CELLULE 1 : MONTAGE GOOGLE DRIVE
# ==========================================

from google.colab import drive
import os

print("="*70)
print("💾 MONTAGE GOOGLE DRIVE")
print("="*70)

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
    print("✅ Google Drive monté")
else:
    print("✅ Google Drive déjà monté")

print("="*70)

💾 MONTAGE GOOGLE DRIVE
Mounted at /content/drive
✅ Google Drive monté


In [ ]:
# ==========================================
# CELLULE 2 : INSTALLATION BARF
# ==========================================
import os
import sys

print("="*70)
print("🔧 INSTALLATION DE L'ENVIRONNEMENT BARF")
print("="*70)

WORKSPACE = '/content/barf_workspace'
BARF_DIR = os.path.join(WORKSPACE, 'bundle-adjusting-NeRF')

os.makedirs(WORKSPACE, exist_ok=True)

if not os.path.exists(BARF_DIR):
    print("\n📥 Clonage du dépôt BARF...")
    os.chdir(WORKSPACE)
    !git clone https://github.com/chenhsuanlin/bundle-adjusting-NeRF.git
    print("✅ Dépôt cloné")
else:
    print("✅ Dépôt BARF déjà présent")

os.chdir(BARF_DIR)
print(f"📂 Répertoire : {os.getcwd()}")

# Installation des dépendances
print("\n📦 Installation des dépendances...")
!pip install -q numpy matplotlib pillow tqdm tensorboard easydict pyyaml opencv-python lpips ipdb

# Créer le module SSIM
print("\n🔧 Configuration module SSIM...")
os.makedirs('external', exist_ok=True)

ssim_code = '''"""SSIM implementation"""
import torch
import torch.nn.functional as F
from math import exp

def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()

def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window

def pytorch_ssim(img1, img2, window_size=11, size_average=True):
    channel = img1.size()[1]
    window = create_window(window_size, channel)
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)

    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channel)
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2

    sigma1_sq = F.conv2d(img1*img1, window, padding=window_size//2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding=window_size//2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding=window_size//2, groups=channel) - mu1_mu2

    C1 = 0.01**2
    C2 = 0.03**2
    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)
'''

with open('external/pohsun_ssim.py', 'w') as f:
    f.write(ssim_code)
with open('external/__init__.py', 'w') as f:
    f.write('')

print("✅ Module SSIM créé")

# Patcher options.py
print("\n🔧 Patch de options.py...")
with open('options.py', 'r') as f:
    content = f.read()

content = content.replace(
    'override = input("override? (y/n) ")',
    'override = "y"  # Auto-patched for Colab'
)
content = content.replace(
    'retry = input("visdom port ({}) not open, retry? (y/n) ".format(opt.visdom.port))',
    'retry = "n"  # Auto-patched for Colab'
)

with open('options.py', 'w') as f:
    f.write(content)

print("✅ options.py patché")

# Configuration YAML
print("\n📄 Configuration YAML...")
import yaml

yaml_path = 'options/barf_blender.yaml'
with open(yaml_path, 'r') as f:
    config_yaml = yaml.safe_load(f)

config_yaml['barf_c2f'] = [0.1, 0.5]
config_yaml['visdom'] = False

with open(yaml_path, 'w') as f:
    yaml.dump(config_yaml, f, default_flow_style=False)

print("✅ YAML configuré")

print("\n" + "="*70)
print("✅ INSTALLATION TERMINÉE")
print("="*70)

🔧 INSTALLATION DE L'ENVIRONNEMENT BARF

📥 Clonage du dépôt BARF...
Cloning into 'bundle-adjusting-NeRF'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 102 (delta 35), reused 26 (delta 26), pack-reused 47 (from 1)
Receiving objects: 100% (102/102), 190.68 KiB | 21.19 MiB/s, done.
Resolving deltas: 100% (54/54), done.
✅ Dépôt cloné
📂 Répertoire : /content/barf_workspace/bundle-adjusting-NeRF

📦 Installation des dépendances...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s eta 0:00:00

🔧 Configuration module SSIM...
✅ Module SSIM créé

🔧 Patch de options.py...
✅ options.py patché

📄 Configuration YAML...
✅ YAML configuré

✅ INSTALLATION TERMINÉE


In [ ]:
# ==========================================
# CELLULE 3 : LANCER LE MONITORING (SIMPLIFIÉ)
# ==========================================
import subprocess
import os
import re

print("="*70)
print("🔄 CONFIGURATION DU MONITORING")
print("="*70)

# === DÉTECTION DU GROUPE ACTUEL ===
# Depuis votre log, le groupe est : vision_20251207_0455
GROUP_NAME = "vision_20251207_0455"  # À remplacer par votre groupe actuel
print(f"✅ Groupe détecté : {GROUP_NAME}")

# Chemins
LOCAL_CHECKPOINTS = f"/content/barf/output/{GROUP_NAME}/training"
DRIVE_CHECKPOINTS = "/content/drive/MyDrive/checkpoints_reel"

os.makedirs(DRIVE_CHECKPOINTS, exist_ok=True)
print(f"📁 Dossier Drive : {DRIVE_CHECKPOINTS}")
print(f"📂 Source checkpoints : {LOCAL_CHECKPOINTS}")
print("="*70)

# Script de surveillance SIMPLIFIÉ
monitor_script = f'''import time
import shutil
import os
import re

GROUP_NAME = "{GROUP_NAME}"
LOCAL_CHECKPOINTS = "/content/barf/output/{{GROUP_NAME}}/training"
DRIVE_CHECKPOINTS = "/content/drive/MyDrive/checkpoints_reel"

os.makedirs(DRIVE_CHECKPOINTS, exist_ok=True)

saved_files = set()
print("="*70)
print("🔄 SURVEILLANCE CHECKPOINTS")
print("="*70)
print(f"Groupe: {{GROUP_NAME}}")
print(f"Source: {{LOCAL_CHECKPOINTS}}")
print(f"Destination: {{DRIVE_CHECKPOINTS}}")
print("="*70)

def should_save_checkpoint(filename):
    """Détermine si le checkpoint doit être sauvegardé"""
    # Toujours sauvegarder latest.ckpt
    if filename == "latest.ckpt":
        return True, "latest"

    # Sauvegarder tous les 100 itérations jusqu'à 1000
    # Puis tous les 1000 itérations
    numbers = re.findall(r'\\d+', filename)
    if numbers:
        iteration = int(numbers[-1])
        if iteration <= 1000 and iteration % 100 == 0:
            return True, f"it={{iteration}}"
        elif iteration > 1000 and iteration % 1000 == 0:
            return True, f"it={{iteration}}"

    return False, "skip"

# Attendre que le dossier existe
max_wait = 300  # 5 minutes
wait_time = 0
while not os.path.exists(LOCAL_CHECKPOINTS) and wait_time < max_wait:
    time.sleep(10)
    wait_time += 10
    if wait_time % 30 == 0:
        print(f"⏳ Attente... ({{wait_time}}s)")

if not os.path.exists(LOCAL_CHECKPOINTS):
    print("❌ Dossier non trouvé")
    exit(1)

print("✅ Dossier trouvé, surveillance démarrée")

while True:
    try:
        if not os.path.exists(LOCAL_CHECKPOINTS):
            time.sleep(30)
            continue

        for f in os.listdir(LOCAL_CHECKPOINTS):
            if f.endswith('.ckpt') and f not in saved_files:
                should_save, reason = should_save_checkpoint(f)

                if should_save:
                    src = os.path.join(LOCAL_CHECKPOINTS, f)
                    dst = os.path.join(DRIVE_CHECKPOINTS, f)

                    try:
                        shutil.copy2(src, dst)
                        size_mb = os.path.getsize(dst) / (1024 * 1024)
                        print(f"💾 {{f}} ({{size_mb:.1f}} MB) - {{reason}}")
                    except Exception as e:
                        print(f"❌ Erreur: {{e}}")

                saved_files.add(f)

        time.sleep(30)

    except KeyboardInterrupt:
        print("\\n🛑 Surveillance arrêtée")
        break
    except Exception as e:
        print(f"⚠️ Erreur: {{e}}")
        time.sleep(60)
'''

# Écrire le script
monitor_script_path = '/content/monitor_checkpoints.py'
with open(monitor_script_path, 'w') as f:
    f.write(monitor_script)

print(f"📄 Script créé : {monitor_script_path}")

# Arrêter les anciens processus de monitoring
import psutil
for proc in psutil.process_iter(['pid', 'name', 'cmdline']):
    try:
        if proc.info['cmdline'] and any(x in ' '.join(proc.info['cmdline']) for x in ['monitor_checkpoints', 'monitor_current']):
            print(f"⚠️ Arrêt de l'ancien monitoring (PID: {proc.info['pid']})")
            proc.terminate()
            try:
                proc.wait(timeout=3)
            except:
                pass
    except:
        pass

# Lancer en arrière-plan
process = subprocess.Popen(['python', monitor_script_path])
print(f"✅ Nouveau monitoring lancé (PID: {process.pid})")
print("="*70)
print("📝 Le monitoring sauvegardera :")
print("   - latest.ckpt (toujours)")
print("   - Tous les 100 itérations jusqu'à 1000")
print("   - Puis tous les 1000 itérations")
print("="*70)

# Vérification immédiate
print("🔍 VÉRIFICATION IMMÉDIATE :")
print(f"Local : {os.path.exists(LOCAL_CHECKPOINTS)}")
print(f"Drive : {os.path.exists(DRIVE_CHECKPOINTS)}")
if os.path.exists(LOCAL_CHECKPOINTS):
    files = [f for f in os.listdir(LOCAL_CHECKPOINTS) if f.endswith('.ckpt')]
    print(f"Checkpoints locaux : {len(files)}")
print("="*70)

🔄 CONFIGURATION DU MONITORING
✅ Groupe détecté : vision_20251207_0455
📁 Dossier Drive : /content/drive/MyDrive/checkpoints_reel
📂 Source checkpoints : /content/barf/output/vision_20251207_0455/training
📄 Script créé : /content/monitor_checkpoints.py
✅ Nouveau monitoring lancé (PID: 1114)
📝 Le monitoring sauvegardera :
   - latest.ckpt (toujours)
   - Tous les 100 itérations jusqu'à 1000
   - Puis tous les 1000 itérations
🔍 VÉRIFICATION IMMÉDIATE :
Local : False
Drive : True


In [ ]:
# ==========================================
# LANCER LA SURVEILLANCE DES CHECKPOINTS
# ==========================================
import subprocess
import time
import os

print("="*70)
print("🚀 LANCEMENT DU SYSTÈME DE SURVEILLANCE")
print("="*70)

# Vérifier que le script de monitoring existe
monitor_script = '/content/monitor_checkpoints.py'
if not os.path.exists(monitor_script):
    print("❌ Script de monitoring non trouvé!")
    print("⚠️ Vous devez d'abord exécuter la cellule qui crée le script")
else:
    # Lancer le monitoring en arrière-plan
    process = subprocess.Popen(
        ['python', '-u', monitor_script],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )

    # Attendre un peu pour vérifier le démarrage
    time.sleep(3)

    # Vérifier que le processus est toujours actif
    if process.poll() is None:
        print(f"✅ Surveillance active (PID: {process.pid})")
        print("="*70)
        print("📝 Politique de sauvegarde :")
        print("   • latest.ckpt : À chaque mise à jour")
        print("   • it_100, it_200... it_1000 : Tous les 100 itérations")
        print("   • it_2000, it_3000... : Tous les 1000 itérations")
        print("="*70)
        print(f"📍 Destination : /content/drive/MyDrive/checkpoints_reel")
        print("="*70)
        print("💡 Le monitoring tourne en arrière-plan")
        print("   Les logs apparaîtront au fur et à mesure des sauvegardes")
    else:
        print("❌ Le processus s'est arrêté immédiatement")
        stdout, stderr = process.communicate()
        if stderr:
            print(f"Erreur: {stderr}")
        if stdout:
            print(f"Output: {stdout}")

print("\n🔍 Vérification des chemins :")
local_path = "/content/barf/output"
drive_path = "/content/drive/MyDrive/checkpoints_reel"

print(f"   Local BARF : {os.path.exists(local_path)} - {local_path}")
print(f"   Drive dest : {os.path.exists(drive_path)} - {drive_path}")

if os.path.exists(local_path):
    groups = [d for d in os.listdir(local_path) if os.path.isdir(os.path.join(local_path, d))]
    if groups:
        print(f"   Groupes trouvés : {', '.join(groups)}")
print("="*70)

🚀 LANCEMENT DU SYSTÈME DE SURVEILLANCE
✅ Surveillance active (PID: 1156)
📝 Politique de sauvegarde :
   • latest.ckpt : À chaque mise à jour
   • it_100, it_200... it_1000 : Tous les 100 itérations
   • it_2000, it_3000... : Tous les 1000 itérations
📍 Destination : /content/drive/MyDrive/checkpoints_reel
💡 Le monitoring tourne en arrière-plan
   Les logs apparaîtront au fur et à mesure des sauvegardes

🔍 Vérification des chemins :
   Local BARF : False - /content/barf/output
   Drive dest : True - /content/drive/MyDrive/checkpoints_reel


In [ ]:
"""
==========================================
BARF VISION - ENTRAÎNEMENT AVEC CHECKPOINTS LOCAUX
Sauvegarde locale + Reprise automatique
==========================================
"""

import os
import json
import shutil
import subprocess
import sys
import numpy as np
import cv2
import struct
from datetime import datetime
from google.colab import drive

print("="*70)
print("🚀 BARF VISION - CHECKPOINTS LOCAUX + REPRISE AUTO")
print("="*70)

# ==========================================
# CONFIGURATION
# ==========================================
print("\n[1/6] 📁 CONFIGURATION")
print("-"*70)

# Monter Google Drive
drive.mount('/content/drive', force_remount=True)

# Chemins
VISION_PATH = '/content/drive/MyDrive/vision'
BARF_PATH = '/content/barf'
DATASET_ROOT = '/content/drive/MyDrive/vision/images'
SCENE_NAME = 'vision_scene'

# Dossiers de sortie LOCAUX (dans /content)
LOCAL_OUTPUT = '/content/barf_output'
LOCAL_CHECKPOINTS = os.path.join(LOCAL_OUTPUT, 'checkpoints')

# Dossiers Drive (pour backup final uniquement)
DRIVE_ROOT = '/content/drive/MyDrive/barf_outputs'
DRIVE_RESULTS = os.path.join(DRIVE_ROOT, 'results')

for folder in [LOCAL_OUTPUT, LOCAL_CHECKPOINTS, DRIVE_RESULTS]:
    os.makedirs(folder, exist_ok=True)

print(f"✅ Chemins configurés")
print(f"   Vision           : {VISION_PATH}")
print(f"   BARF             : {BARF_PATH}")
print(f"   Dataset          : {DATASET_ROOT}/{SCENE_NAME}")
print(f"   Checkpoints local: {LOCAL_CHECKPOINTS}")
print(f"   Résultats Drive  : {DRIVE_RESULTS}")

# ==========================================
# INSTALLATION BARF
# ==========================================
print("\n[2/6] 🔧 INSTALLATION BARF")
print("-"*70)

if not os.path.exists(BARF_PATH):
    print("📥 Clone du repository BARF...")
    subprocess.run([
        'git', 'clone',
        'https://github.com/chenhsuanlin/bundle-adjusting-NeRF.git',
        BARF_PATH
    ], check=True)

    print("📦 Installation des dépendances...")
    os.chdir(BARF_PATH)

    # PyTorch
    subprocess.run([
        'pip', 'install', 'torch', 'torchvision',
        '--index-url', 'https://download.pytorch.org/whl/cu118'
    ], check=True)

    # Requirements
    subprocess.run(['pip', 'install', '-r', 'requirements.txt'], check=True)

    # Submodules (pour SSIM)
    subprocess.run(['git', 'submodule', 'update', '--init', '--recursive'], check=True)

    # Fix SSIM module
    print("🔧 Configuration du module SSIM...")
    ssim_path = os.path.join(BARF_PATH, 'external', 'pohsun_ssim')
    if os.path.exists(ssim_path):
        # Créer __init__.py si nécessaire
        init_file = os.path.join(ssim_path, '__init__.py')
        if not os.path.exists(init_file):
            with open(init_file, 'w') as f:
                f.write('from .pytorch_ssim import *\n')
            print("   ✅ __init__.py créé pour SSIM")

        # Vérifier que pytorch_ssim.py existe
        ssim_file = os.path.join(ssim_path, 'pytorch_ssim.py')
        if os.path.exists(ssim_file):
            print("   ✅ Module SSIM configuré")
        else:
            print("   ⚠️ pytorch_ssim.py manquant, téléchargement...")
            subprocess.run([
                'wget', '-O', ssim_file,
                'https://raw.githubusercontent.com/Po-Hsun-Su/pytorch-ssim/master/pytorch_ssim/__init__.py'
            ], check=True)
    else:
        print("   ⚠️ Dossier SSIM non trouvé")

    print("✅ BARF installé")
else:
    os.chdir(BARF_PATH)
    print("✅ BARF déjà présent")

    # Vérifier et réparer SSIM si nécessaire
    print("🔍 Vérification du module SSIM...")
    ssim_path = os.path.join(BARF_PATH, 'external', 'pohsun_ssim')
    ssim_file = os.path.join(ssim_path, 'pytorch_ssim.py')
    init_file = os.path.join(ssim_path, '__init__.py')

    if not os.path.exists(ssim_file) or not os.path.exists(init_file):
        print("🔧 Réparation du module SSIM...")
        os.makedirs(ssim_path, exist_ok=True)

        # Télécharger pytorch_ssim.py
        if not os.path.exists(ssim_file):
            subprocess.run([
                'wget', '-O', ssim_file,
                'https://raw.githubusercontent.com/Po-Hsun-Su/pytorch-ssim/master/pytorch_ssim/__init__.py'
            ], check=True)
            print("   ✅ pytorch_ssim.py téléchargé")

        # Créer __init__.py
        if not os.path.exists(init_file):
            with open(init_file, 'w') as f:
                f.write('from .pytorch_ssim import *\n')
            print("   ✅ __init__.py créé")
    else:
        print("   ✅ Module SSIM OK")

# ==========================================
# FONCTIONS DE LECTURE COLMAP
# ==========================================

def read_colmap_bin_file(bin_file):
    """Lit un fichier binaire COLMAP"""
    if not os.path.exists(bin_file):
        return None

    data = []
    with open(bin_file, 'rb') as f:
        num_elements_bytes = f.read(8)
        if len(num_elements_bytes) < 8:
            return None
        num_elements = struct.unpack('Q', num_elements_bytes)[0]

        print(f"   📊 {num_elements} images à lire")

        for i in range(num_elements):
            try:
                image_id = struct.unpack('I', f.read(4))[0]
                quat = struct.unpack('dddd', f.read(8*4))
                qw, qx, qy, qz = quat
                trans = struct.unpack('ddd', f.read(8*3))
                tx, ty, tz = trans
                camera_id = struct.unpack('I', f.read(4))[0]

                name_bytes = b''
                while True:
                    char = f.read(1)
                    if char == b'\x00' or not char:
                        break
                    name_bytes += char

                name = name_bytes.decode('utf-8', errors='ignore')
                num_points2D = struct.unpack('Q', f.read(8))[0]
                f.seek(num_points2D * 24, 1)

                data.append({
                    'image_id': image_id,
                    'qw': qw, 'qx': qx, 'qy': qy, 'qz': qz,
                    'tx': tx, 'ty': ty, 'tz': tz,
                    'camera_id': camera_id,
                    'name': name
                })

                if (i + 1) % 100 == 0:
                    print(f"   📖 {i + 1}/{num_elements} images lues...")

            except Exception as e:
                print(f"   ⚠️ Erreur à l'image {i}: {e}")
                break

    print(f"   ✅ {len(data)} images lues avec succès")
    return data

def parse_colmap_txt_file(txt_file):
    """Parse un fichier texte COLMAP"""
    if not os.path.exists(txt_file):
        return None

    print(f"   📖 Lecture du fichier texte: {txt_file}")

    images_section = False
    images_data = []

    with open(txt_file, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        line = line.strip()

        if not line or line.startswith('#'):
            continue

        if line == "IMAGES":
            images_section = True
            continue
        elif line in ["CAMERAS", "POINTS3D"]:
            images_section = False
            continue

        if images_section:
            parts = line.split()
            if len(parts) >= 10:
                try:
                    image_data = {
                        'image_id': int(parts[0]),
                        'qw': float(parts[1]),
                        'qx': float(parts[2]),
                        'qy': float(parts[3]),
                        'qz': float(parts[4]),
                        'tx': float(parts[5]),
                        'ty': float(parts[6]),
                        'tz': float(parts[7]),
                        'camera_id': int(parts[8]),
                        'name': parts[9]
                    }
                    images_data.append(image_data)
                except ValueError as e:
                    print(f"   ⚠️ Erreur parsing ligne {i}: {e}")
                    continue

    print(f"   ✅ {len(images_data)} images extraites")
    return images_data
# ==========================================
# INSTALLATION MANQUANTE - CORRECTION
# ==========================================
print("\n🔧 INSTALLATION DES DÉPENDANCES MANQUANTES")
print("-"*70)

# Installation des packages manquants
missing_packages = ['ipdb', 'tensorboardX', 'lpips', 'easydict']
for package in missing_packages:
    print(f"📦 Installation de {package}...")
    try:
        subprocess.run(['pip', 'install', '-q', package], check=True)
        print(f"   ✅ {package} installé")
    except:
        print(f"   ⚠️ Échec installation {package}")
print("="*70)
print("🔧 INSTALLATION VISDOM")
print("="*70)

!pip install -q visdom
print("✅ visdom installé")

# Optionnel : Désactiver visdom complètement dans la configuration
print("\n⚠️ Alternative : Désactiver visdom dans BARF")
print("   Modifiez options/barf_blender.yaml :")
print("   visdom.port: -1")
print("   visdom.server: 'http://localhost'")
print("="*70)

# ==========================================
# GESTION DES CHECKPOINTS
# ==========================================
print("\n[3/6] 🔍 VÉRIFICATION DES CHECKPOINTS")
print("-"*70)

def find_latest_checkpoint(output_dir):
    """Trouve le dernier checkpoint disponible"""
    if not os.path.exists(output_dir):
        return None, None

    # Chercher tous les groupes
    groups = [d for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]

    if not groups:
        return None, None

    # Trier par date (format: vision_YYYYMMDD_HHMM)
    groups.sort(reverse=True)
    latest_group = groups[0]

    # Chercher le checkpoint dans ce groupe
    group_path = os.path.join(output_dir, latest_group, 'training')
    if not os.path.exists(group_path):
        return None, None

    # Chercher les fichiers .ckpt
    ckpt_files = [f for f in os.listdir(group_path) if f.endswith('.ckpt')]

    if not ckpt_files:
        return latest_group, None

    # Trouver le checkpoint avec l'itération la plus élevée
    max_iter = -1
    latest_ckpt = None

    for ckpt in ckpt_files:
        # Format: it_XXXXX.ckpt ou latest.ckpt
        if ckpt == 'latest.ckpt':
            latest_ckpt = ckpt
            # Lire l'itération depuis le fichier
            ckpt_path = os.path.join(group_path, ckpt)
            import torch
            try:
                checkpoint = torch.load(ckpt_path, map_location='cpu')
                iter_num = checkpoint.get('iter', 0)
                if iter_num > max_iter:
                    max_iter = iter_num
            except:
                pass
        else:
            try:
                iter_num = int(ckpt.split('_')[1].split('.')[0])
                if iter_num > max_iter:
                    max_iter = iter_num
                    latest_ckpt = ckpt
            except:
                pass

    print(f"   ✅ Checkpoint trouvé: {latest_group}/{latest_ckpt}")
    print(f"   📊 Itération: {max_iter}")

    return latest_group, latest_ckpt

# Chercher un checkpoint existant
RESUME_GROUP, RESUME_CKPT = find_latest_checkpoint(os.path.join(BARF_PATH, 'output'))

if RESUME_GROUP and RESUME_CKPT:
    print(f"\n🔄 REPRISE D'ENTRAÎNEMENT DÉTECTÉE")
    print(f"   Groupe: {RESUME_GROUP}")
    print(f"   Checkpoint: {RESUME_CKPT}")
    RESUME_MODE = True
    GROUP_NAME = RESUME_GROUP
else:
    print(f"\n🆕 NOUVEL ENTRAÎNEMENT")
    RESUME_MODE = False
    GROUP_NAME = f'vision_{datetime.now().strftime("%Y%m%d_%H%M")}'

# ==========================================
# CONVERSION DATASET
# ==========================================
print("\n[4/6] 🔄 CONVERSION DATASET")
print("-"*70)

def convert_colmap_to_blender_format(vision_path, output_path):
    """Convertit les données COLMAP au format Blender"""
    os.makedirs(output_path, exist_ok=True)
    images_output = os.path.join(output_path, 'images')
    os.makedirs(images_output, exist_ok=True)

    # Chercher le dossier sparse
    sparse_0_path = None
    possible_paths = [
        os.path.join(vision_path, "sparse", "0"),
        os.path.join(vision_path, "sparse_text"),
        os.path.join(vision_path, "sparse"),
    ]

    for path in possible_paths:
        if os.path.exists(path):
            sparse_0_path = path
            print(f"✅ Dossier sparse trouvé : {path}")
            break

    if not sparse_0_path:
        print("❌ Aucun dossier sparse trouvé !")
        return False

    # Lister les fichiers COLMAP
    print(f"\n📂 Contenu de {sparse_0_path} :")
    colmap_files = {
        'cameras.bin': None,
        'images.bin': None,
        'points3D.bin': None,
        'cameras.txt': None,
        'images.txt': None,
        'points3D.txt': None
    }

    for file in os.listdir(sparse_0_path):
        if file in colmap_files:
            colmap_files[file] = os.path.join(sparse_0_path, file)
            print(f"   ✅ {file}")

    # Lire les données COLMAP
    print("\n🔍 Lecture des données COLMAP...")
    colmap_data = None

    if colmap_files['images.bin'] and os.path.exists(colmap_files['images.bin']):
        print(f"📖 Tentative de lecture binaire: {colmap_files['images.bin']}")
        colmap_data = read_colmap_bin_file(colmap_files['images.bin'])

        if not colmap_data or len(colmap_data) == 0:
            print("   ⚠️ Lecture binaire échouée, tentative texte...")
            if colmap_files['images.txt']:
                colmap_data = parse_colmap_txt_file(colmap_files['images.txt'])
    elif colmap_files['images.txt']:
        colmap_data = parse_colmap_txt_file(colmap_files['images.txt'])

    if not colmap_data:
        print("❌ Aucune donnée COLMAP valide trouvée")
        return False

    print(f"✅ Données COLMAP chargées: {len(colmap_data)} images")

    # Chercher le dossier des images
    images_dir = None
    possible_image_dirs = [
        os.path.join(vision_path, "images"),
        os.path.join(vision_path, "..", "images"),
        "/content/images",
        os.path.dirname(vision_path)
    ]

    for img_dir in possible_image_dirs:
        if os.path.exists(img_dir) and len(os.listdir(img_dir)) > 0:
            images_dir = img_dir
            image_count = len([f for f in os.listdir(img_dir)
                             if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"✅ Dossier images trouvé : {img_dir} ({image_count} images)")
            break

    if not images_dir:
        print("❌ Dossier images non trouvé")
        return False

    # Traiter les images
    image_extensions = ('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(image_extensions)]

    image_map = {}
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        image_map[base_name.lower()] = os.path.join(images_dir, img_file)

    print("\n🔄 Conversion des images en PNG RGBA...")
    images_processed = 0
    actual_images = []

    for i, img_info in enumerate(colmap_data):
        img_name = img_info.get('name', f'image_{i:04d}')
        base_name = os.path.splitext(img_name)[0]

        img_found = False
        src_path = None

        if base_name.lower() in image_map:
            src_path = image_map[base_name.lower()]
            img_found = True

        if not img_found:
            for img_file in image_files:
                if base_name in img_file or img_name in img_file:
                    src_path = os.path.join(images_dir, img_file)
                    img_found = True
                    break

        if img_found and src_path:
            dst_path = os.path.join(images_output, f"{base_name}.png")

            try:
                img = cv2.imread(src_path, cv2.IMREAD_UNCHANGED)
                if img is None:
                    continue

                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGBA)
                elif img.shape[2] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)

                h, w = img.shape[:2]
                max_size = 800
                if max(h, w) > max_size:
                    scale = max_size / max(h, w)
                    new_w = int(w * scale)
                    new_h = int(h * scale)
                    img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

                cv2.imwrite(dst_path, img)
                actual_images.append((i, img_name, dst_path, w, h))
                images_processed += 1

                if i < 5 or i % 50 == 0:
                    print(f"   ✅ Image {i:04d}: {os.path.basename(src_path)} → {base_name}.png")

            except Exception as e:
                print(f"   ⚠️ Erreur traitement {img_name}: {e}")

    print(f"\n🎨 {images_processed} images traitées")

    # Créer transforms.json
    transforms = {
        "camera_angle_x": 0.6911112070083618,
        "frames": []
    }

    for idx, img_name, img_path, img_width, img_height in actual_images:
        if idx < len(colmap_data):
            img_info = colmap_data[idx]

            qw = img_info.get('qw', 1.0)
            qx = img_info.get('qx', 0.0)
            qy = img_info.get('qy', 0.0)
            qz = img_info.get('qz', 0.0)

            tx = img_info.get('tx', 0.0)
            ty = img_info.get('ty', 0.0)
            tz = img_info.get('tz', 0.0)

            norm = np.sqrt(qw*qw + qx*qx + qy*qy + qz*qz)
            if norm > 0:
                qw, qx, qy, qz = qw/norm, qx/norm, qy/norm, qz/norm

            R = np.array([
                [1 - 2*qy*qy - 2*qz*qz, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
                [2*qx*qy + 2*qz*qw, 1 - 2*qx*qx - 2*qz*qz, 2*qy*qz - 2*qx*qw],
                [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx*qx - 2*qy*qy]
            ])

            transform = np.eye(4)
            transform[:3, :3] = R
            transform[:3, 3] = [tx, ty, tz]

            base_name = os.path.splitext(os.path.basename(img_path))[0]
            frame_path = f"./images/{base_name}"

            transforms['frames'].append({
                "file_path": frame_path,
                "transform_matrix": transform.tolist()
            })

    # Normaliser les poses
    print(f"\n📐 Normalisation des poses...")
    if transforms['frames']:
        positions = []
        for frame in transforms['frames']:
            transform = np.array(frame['transform_matrix'])
            position = transform[:3, 3]
            positions.append(position)

        positions = np.array(positions)
        center = np.mean(positions, axis=0)
        max_dist = np.max(np.linalg.norm(positions - center, axis=1))

        if max_dist > 0:
            scale_factor = 1.5 / max_dist

            for frame in transforms['frames']:
                transform = np.array(frame['transform_matrix'])
                transform[:3, 3] = (transform[:3, 3] - center) * scale_factor
                frame['transform_matrix'] = transform.tolist()

    # Sauvegarder
    transforms_path = os.path.join(output_path, 'transforms_train.json')
    with open(transforms_path, 'w') as f:
        json.dump(transforms, f, indent=2)

    # Créer val et test
    val_indices = [i for i in range(len(transforms['frames'])) if i % 5 == 0]
    val_transforms = {
        "camera_angle_x": transforms["camera_angle_x"],
        "frames": [transforms['frames'][i] for i in val_indices]
    }

    test_indices = [i for i in range(len(transforms['frames'])) if i % 10 == 0]
    test_transforms = {
        "camera_angle_x": transforms["camera_angle_x"],
        "frames": [transforms['frames'][i] for i in test_indices]
    }

    with open(os.path.join(output_path, 'transforms_val.json'), 'w') as f:
        json.dump(val_transforms, f, indent=2)

    with open(os.path.join(output_path, 'transforms_test.json'), 'w') as f:
        json.dump(test_transforms, f, indent=2)

    print(f"\n✅ Dataset créé : {output_path}")
    print(f"   Train: {len(transforms['frames'])}")
    print(f"   Val  : {len(val_transforms['frames'])}")
    print(f"   Test : {len(test_transforms['frames'])}")

    return True

# Conversion (seulement si nouveau training)
scene_path = os.path.join(DATASET_ROOT, SCENE_NAME)
if not RESUME_MODE:
    if os.path.exists(scene_path):
        shutil.rmtree(scene_path)

    success = convert_colmap_to_blender_format(VISION_PATH, scene_path)

    if not success:
        print("❌ Conversion échouée")
        sys.exit(1)
else:
    print("♻️ Mode reprise: dataset existant réutilisé")

# ==========================================
# MODIFICATION DU CODE SOURCE POUR DÉSACTIVER VISDOM
# ==========================================
print("\n[4/5] ⚙️ PATCH BARF POUR COLAB")
print("-"*70)

# Modifier directement le fichier base.py pour éviter l'interaction
base_py_path = os.path.join(BARF_PATH, 'model', 'base.py')
if os.path.exists(base_py_path):
    with open(base_py_path, 'r') as f:
        content = f.read()

    # Remplacer la fonction setup_visualizer pour éviter input()
    old_code = '''    def setup_visualizer(self,opt):
        # [not implemented; subclasses should override if needed]
        if opt.visdom.enabled:
            # attempt connection (if not open)
            if not visdom_check_connection(opt.visdom.port):
                retry = input("visdom port ({}) not open, retry? (y/n) ".format(opt.visdom.port))
                if retry=="y":
                    visdom_launch(opt.visdom.port)
                    self.visdom = visdom.Visdom(server="http://localhost",port=opt.visdom.port,env=opt.visdom.env)
                else: opt.visdom.enabled = False
            else: self.visdom = visdom.Visdom(server="http://localhost",port=opt.visdom.port,env=opt.visdom.env)'''

    new_code = '''    def setup_visualizer(self,opt):
        # [not implemented; subclasses should override if needed]
        if opt.visdom.enabled:
            # attempt connection (if not open)
            if not visdom_check_connection(opt.visdom.port):
                # AUTO-RESPONSE pour Colab - pas d'interaction
                print("visdom port ({}) not open, automatically disabling for Colab".format(opt.visdom.port))
                opt.visdom.enabled = False
            else: self.visdom = visdom.Visdom(server="http://localhost",port=opt.visdom.port,env=opt.visdom.env)'''

    if old_code in content:
        content = content.replace(old_code, new_code)
        with open(base_py_path, 'w') as f:
            f.write(content)
        print("✅ base.py patché pour désactiver Visdom automatiquement")
    else:
        print("⚠️ Impossible de trouver le code à patcher dans base.py")
else:
    print("⚠️ Fichier base.py non trouvé")


# ==========================================
# ENTRAÎNEMENT
# ==========================================
print("\n[6/6] 🚀 ENTRAÎNEMENT")
print("="*70)

MAX_ITER = 50000

print(f"🎯 Paramètres :")
print(f"   Mode        : {'REPRISE' if RESUME_MODE else 'NOUVEAU'}")
print(f"   Groupe      : {GROUP_NAME}")
print(f"   Itérations  : {MAX_ITER}")
if RESUME_MODE:
    print(f"   Checkpoint  : {RESUME_CKPT}")
print("="*70)

# Commande d'entraînement
train_cmd = [
    'python', '-u', 'train.py',
    '--yaml=barf_blender',
    '--model=barf',
    f'--group={GROUP_NAME}',
    f'--name=training',
    f'--data.scene={SCENE_NAME}',
    f'--data.root={DATASET_ROOT}',
    f'--max_iter={MAX_ITER}',
    '--visdom.port=-1',
    '--visdom.server=http://localhost',
    '--freq.ckpt=1000',
    '--freq.val=1000',
    '--freq.vis=1000',
]

# Ajouter resume si checkpoint existant
if RESUME_MODE and RESUME_CKPT:
    train_cmd.append('--resume')
    train_cmd.append(f'--load={RESUME_CKPT.replace(".ckpt", "")}')

print(f"\n📊 Commande: {' '.join(train_cmd)}")
print("-"*70)

try:
    process = subprocess.Popen(
        train_cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    for line in process.stdout:
        print(line, end='', flush=True)

    process.wait()

    if process.returncode == 0:
        print("\n" + "="*70)
        print("✅ ENTRAÎNEMENT TERMINÉ")
        print("="*70)

        # Copier les résultats vers Drive (backup final)
        output_src = os.path.join(BARF_PATH, 'output', GROUP_NAME)
        output_dst = os.path.join(DRIVE_RESULTS, GROUP_NAME)

        if os.path.exists(output_src):
            print(f"\n💾 Sauvegarde finale sur Drive...")
            shutil.copytree(output_src, output_dst, dirs_exist_ok=True)
            print(f"✅ Résultats sauvegardés : {output_dst}")

except KeyboardInterrupt:
    print("\n⚠️ Interrompu par l'utilisateur")
    print("💡 Les checkpoints sont sauvegardés localement")
    print("   Relancez le script pour reprendre l'entraînement")
    if 'process' in locals():
        process.terminate()
        process.wait()

except Exception as e:
    print(f"\n❌ Erreur : {e}")

print("\n" + "="*70)
print("🎉 PROCESSUS COMPLET")
print("="*70)
print(f"\n📊 RÉSUMÉ :")
print(f"   Dataset    : {scene_path}")
print(f"   Groupe     : {GROUP_NAME}")
print(f"   Checkpoints: {os.path.join(BARF_PATH, 'output', GROUP_NAME)}")
print(f"   Backup Drive: {DRIVE_RESULTS}/{GROUP_NAME}")
print("="*70)

🚀 BARF VISION - CHECKPOINTS LOCAUX + REPRISE AUTO

[1/6] 📁 CONFIGURATION
----------------------------------------------------------------------
Mounted at /content/drive
✅ Chemins configurés
   Vision           : /content/drive/MyDrive/vision
   BARF             : /content/barf
   Dataset          : /content/drive/MyDrive/vision/images/vision_scene
   Checkpoints local: /content/barf_output/checkpoints
   Résultats Drive  : /content/drive/MyDrive/barf_outputs/results

[2/6] 🔧 INSTALLATION BARF
----------------------------------------------------------------------
✅ BARF déjà présent
🔍 Vérification du module SSIM...
🔧 Réparation du module SSIM...
   ✅ pytorch_ssim.py téléchargé
   ✅ __init__.py créé

🔧 INSTALLATION DES DÉPENDANCES MANQUANTES
----------------------------------------------------------------------
📦 Installation de ipdb...
   ✅ ipdb installé
📦 Installation de tensorboardX...
   ✅ tensorboardX installé
📦 Installation de lpips...
   ✅ lpips installé
📦 Installation de easydic

....

In [ ]:
"""
==========================================
BARF VISION - VERSION FINALE CORRIGÉE
Configuration 6h pour 93 images - TOUS BUGS CORRIGÉS
Incluant fix Visdom + TensorBoard définitif
==========================================
"""

import os
import json
import shutil
import subprocess
import sys
import random
import numpy as np
import cv2
import struct
from datetime import datetime
from google.colab import drive

print("="*70)
print("🚀 BARF VISION - VERSION FINALE CORRIGÉE (Visdom + TensorBoard)")
print("="*70)

# ==========================================
# 1. MONTAGE DRIVE & CONFIGURATION
# ==========================================
print("\n[1/7] 📁 CONFIGURATION")
print("-"*70)

drive.mount('/content/drive', force_remount=True)

# Chemins
VISION_PATH = '/content/drive/MyDrive/vision'
BARF_PATH = '/content/barf'
DATASET_ROOT = '/content/drive/MyDrive/vision/images'
SCENE_NAME = 'vision_scene'

DRIVE_ROOT = '/content/drive/MyDrive/barf_outputs'
DRIVE_RESULTS = os.path.join(DRIVE_ROOT, 'results')

os.makedirs(DRIVE_RESULTS, exist_ok=True)

print(f"✅ Chemins configurés")
print(f"   Vision  : {VISION_PATH}")
print(f"   Dataset : {DATASET_ROOT}/{SCENE_NAME}")
print(f"   Résultats: {DRIVE_RESULTS}")

# ==========================================
# 2. INSTALLATION BARF
# ==========================================
print("\n[2/7] 🔧 INSTALLATION BARF")
print("-"*70)

if not os.path.exists(BARF_PATH):
    print("📥 Clonage du repository BARF...")
    subprocess.run([
        'git', 'clone',
        'https://github.com/chenhsuanlin/bundle-adjusting-NeRF.git',
        BARF_PATH
    ], check=True, capture_output=True)

    os.chdir(BARF_PATH)

    print("📦 Installation des dépendances...")

    # PyTorch
    try:
        import torch
        print(f"   ✅ PyTorch {torch.__version__} déjà installé")
    except ImportError:
        subprocess.run([
            'pip', 'install', '-q', 'torch', 'torchvision',
            '--index-url', 'https://download.pytorch.org/whl/cu118'
        ], check=True)
        print("   ✅ PyTorch installé")

    # Requirements BARF
    if os.path.exists('requirements.txt'):
        subprocess.run(['pip', 'install', '-q', '-r', 'requirements.txt'], check=True)
        print("   ✅ Requirements installés")

    # Visdom (EXPLICITE)
    print("📦 Installation visdom...")
    subprocess.run(['pip', 'install', '-q', 'visdom'], check=True)
    print("   ✅ Visdom installé")

    # Submodules
    subprocess.run(['git', 'submodule', 'update', '--init', '--recursive'],
                   check=True, capture_output=True)
    print("   ✅ Submodules initialisés")

    print("✅ BARF installé")
else:
    os.chdir(BARF_PATH)
    print("✅ BARF déjà présent")

    # S'assurer que visdom est installé
    try:
        import visdom
        print("   ✅ Visdom déjà installé")
    except ImportError:
        print("📦 Installation visdom...")
        subprocess.run(['pip', 'install', '-q', 'visdom'], check=True)
        print("   ✅ Visdom installé")

# Configuration SSIM
print("\n🔧 Configuration module SSIM...")
ssim_path = os.path.join(BARF_PATH, 'external', 'pohsun_ssim')
os.makedirs(ssim_path, exist_ok=True)

ssim_file = os.path.join(ssim_path, 'pytorch_ssim.py')
init_file = os.path.join(ssim_path, '__init__.py')

if not os.path.exists(ssim_file):
    subprocess.run([
        'wget', '-q', '-O', ssim_file,
        'https://raw.githubusercontent.com/Po-Hsun-Su/pytorch-ssim/master/pytorch_ssim/__init__.py'
    ], check=True)
    print("   ✅ pytorch_ssim.py téléchargé")

if not os.path.exists(init_file):
    with open(init_file, 'w') as f:
        f.write('from .pytorch_ssim import *\n')
    print("   ✅ __init__.py créé")

# ==========================================
# 3. PATCH VISDOM + TENSORBOARD COMPLET
# ==========================================
print("\n[3/7] 🔧 DÉSACTIVATION VISDOM + CONFIGURATION TENSORBOARD")
print("-"*70)

# Patch 1: Modifier base.py (VISDOM + TENSORBOARD)
base_py_path = os.path.join(BARF_PATH, 'model', 'base.py')
if os.path.exists(base_py_path):
    with open(base_py_path, 'r') as f:
        content = f.read()

    # Remplacer TOUTE la fonction setup_visualizer
    import re

    new_function = '''def setup_visualizer(self,opt):
        # Visdom complètement désactivé pour Colab
        opt.visdom.enabled = False
        self.vis = None

        # Initialiser TensorBoard obligatoirement
        from torch.utils.tensorboard import SummaryWriter
        import os

        tb_path = os.path.join(opt.output_path, 'tensorboard')
        os.makedirs(tb_path, exist_ok=True)
        self.tb = SummaryWriter(log_dir=tb_path)

        print("✅ Visdom désactivé (mode Colab)")
        print(f"✅ TensorBoard initialisé: {tb_path}")'''

    pattern = r'def setup_visualizer\(self,opt\):.*?(?=\n    def |\nclass |\Z)'
    content_modified = re.sub(pattern, new_function, content, flags=re.DOTALL)

    with open(base_py_path, 'w') as f:
        f.write(content_modified)
    print("   ✅ base.py patché (Visdom + TensorBoard)")

# Patch 2: Modifier util.py pour éviter check_socket_open avec port -1
util_py_path = os.path.join(BARF_PATH, 'util.py')
if os.path.exists(util_py_path):
    with open(util_py_path, 'r') as f:
        content = f.read()

    # Ajouter une vérification au début de check_socket_open
    old_check = 'def check_socket_open(hostname,port):'
    new_check = '''def check_socket_open(hostname,port):
    # Protection pour port invalide (Colab)
    if port < 0 or port > 65535:
        return False'''

    if old_check in content and 'Protection pour port invalide' not in content:
        content = content.replace(old_check, new_check)
        with open(util_py_path, 'w') as f:
            f.write(content)
        print("   ✅ util.py patché")

# Patch 2B: Modifier util_vis.py pour désactiver les appels Visdom
util_vis_path = os.path.join(BARF_PATH, 'util_vis.py')
if os.path.exists(util_vis_path):
    # D'abord, restaurer le fichier original si corrompu
    print("   🔄 Vérification util_vis.py...")

    with open(util_vis_path, 'r') as f:
        content = f.read()

    # Si le fichier est corrompu (syntaxe invalide), le restaurer
    if 'returndef' in content or content.count('if vis is None: return') > 10:
        print("   ⚠️ Fichier corrompu détecté, restauration depuis Git...")
        subprocess.run(
            ['git', 'checkout', 'HEAD', 'util_vis.py'],
            cwd=BARF_PATH,
            capture_output=True
        )
        # Recharger le contenu
        with open(util_vis_path, 'r') as f:
            content = f.read()
        print("   ✅ Fichier restauré")

    # Maintenant appliquer le patch proprement
    lines = content.split('\n')
    new_lines = []
    i = 0

    while i < len(lines):
        line = lines[i]
        new_lines.append(line)

        # Si c'est une définition de fonction vis_*
        if line.strip().startswith('def vis_') and 'vis' in line and ':' in line:
            # Vérifier si on a déjà patché cette fonction
            if i + 1 < len(lines):
                next_line = lines[i + 1]
                if 'if vis is None' not in next_line:
                    # Calculer l'indentation
                    indent = len(line) - len(line.lstrip())
                    # Ajouter la protection avec la bonne indentation
                    new_lines.append(' ' * (indent + 4) + 'if vis is None: return')

        i += 1

    # Écrire le fichier modifié
    with open(util_vis_path, 'w') as f:
        f.write('\n'.join(new_lines))

    print("   ✅ util_vis.py patché")

# Patch 3: Modifier la config YAML pour forcer disabled
yaml_path = os.path.join(BARF_PATH, 'options', 'barf_blender.yaml')
if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        lines = f.readlines()

    # Chercher et modifier la section visdom
    new_lines = []
    in_visdom = False

    for line in lines:
        if 'visdom:' in line:
            in_visdom = True
            new_lines.append(line)
            new_lines.append('    enabled: false\n')
            continue

        if in_visdom and line.strip() and not line.startswith(' '):
            in_visdom = False

        if in_visdom and ('port:' in line or 'enabled:' in line):
            continue  # Skip ces lignes, on les a déjà ajoutées

        new_lines.append(line)

    with open(yaml_path, 'w') as f:
        f.writelines(new_lines)
    print("   ✅ YAML patché")

print("✅ Visdom désactivé + TensorBoard configuré (4 niveaux de protection)")

# ==========================================
# 4. FONCTIONS COLMAP
# ==========================================

def read_colmap_bin_file(bin_file):
    """Lit un fichier binaire COLMAP"""
    if not os.path.exists(bin_file):
        return None

    data = []
    with open(bin_file, 'rb') as f:
        try:
            num_elements = struct.unpack('Q', f.read(8))[0]
            print(f"   📊 {num_elements} images à traiter")

            for i in range(num_elements):
                image_id = struct.unpack('I', f.read(4))[0]
                qw, qx, qy, qz = struct.unpack('dddd', f.read(32))
                tx, ty, tz = struct.unpack('ddd', f.read(24))
                camera_id = struct.unpack('I', f.read(4))[0]

                name_bytes = b''
                while True:
                    char = f.read(1)
                    if char == b'\x00' or not char:
                        break
                    name_bytes += char

                name = name_bytes.decode('utf-8', errors='ignore')
                num_points2D = struct.unpack('Q', f.read(8))[0]
                f.seek(num_points2D * 24, 1)

                data.append({
                    'image_id': image_id,
                    'qw': qw, 'qx': qx, 'qy': qy, 'qz': qz,
                    'tx': tx, 'ty': ty, 'tz': tz,
                    'camera_id': camera_id,
                    'name': name
                })

                if (i + 1) % 50 == 0:
                    print(f"   📖 {i + 1}/{num_elements}...")

        except Exception as e:
            print(f"   ⚠️ Erreur lecture: {e}")
            return None

    return data

def parse_colmap_txt_file(txt_file):
    """Parse un fichier texte COLMAP"""
    if not os.path.exists(txt_file):
        return None

    colmap_data = []
    with open(txt_file, 'r') as f:
        lines = [l.strip() for l in f if l.strip() and not l.startswith('#')]

        for i in range(0, len(lines), 2):
            if i < len(lines):
                parts = lines[i].split()
                if len(parts) >= 10:
                    colmap_data.append({
                        'image_id': int(parts[0]),
                        'qw': float(parts[1]),
                        'qx': float(parts[2]),
                        'qy': float(parts[3]),
                        'qz': float(parts[4]),
                        'tx': float(parts[5]),
                        'ty': float(parts[6]),
                        'tz': float(parts[7]),
                        'camera_id': int(parts[8]),
                        'name': parts[9]
                    })

    return colmap_data if colmap_data else None

# ==========================================
# 5. DÉTECTION CHECKPOINT
# ==========================================
print("\n[4/7] 🔍 VÉRIFICATION CHECKPOINT")
print("-"*70)

def find_latest_checkpoint(output_dir):
    """Trouve le dernier checkpoint"""
    if not os.path.exists(output_dir):
        return None, None

    groups = [d for d in os.listdir(output_dir)
              if os.path.isdir(os.path.join(output_dir, d))
              and d.startswith('vision_')]

    if not groups:
        return None, None

    groups.sort(reverse=True)
    latest_group = groups[0]

    group_path = os.path.join(output_dir, latest_group, 'training')
    if os.path.exists(os.path.join(group_path, 'latest.ckpt')):
        print(f"   ✅ Checkpoint trouvé: {latest_group}/latest.ckpt")
        return latest_group, 'latest'

    return latest_group, None

# Lien symbolique
BARF_OUTPUT = os.path.join(BARF_PATH, 'output')
if os.path.exists(BARF_OUTPUT):
    if os.path.islink(BARF_OUTPUT):
        os.unlink(BARF_OUTPUT)
    else:
        shutil.rmtree(BARF_OUTPUT)

os.symlink(DRIVE_RESULTS, BARF_OUTPUT)
print(f"✅ Lien symbolique: {BARF_OUTPUT} → {DRIVE_RESULTS}")

# Chercher checkpoint
RESUME_GROUP, RESUME_CKPT = find_latest_checkpoint(DRIVE_RESULTS)

if RESUME_GROUP and RESUME_CKPT:
    print(f"🔄 REPRISE DÉTECTÉE")
    print(f"   Groupe: {RESUME_GROUP}")
    RESUME_MODE = True
    GROUP_NAME = RESUME_GROUP
else:
    print(f"🆕 NOUVEL ENTRAÎNEMENT")
    RESUME_MODE = False
    GROUP_NAME = f'vision_{datetime.now().strftime("%Y%m%d_%H%M")}'
    print(f"   Groupe: {GROUP_NAME}")

# ==========================================
# 6. CONVERSION DATASET
# ==========================================
print("\n[5/7] 🔄 CONVERSION DATASET")
print("-"*70)

def convert_colmap_to_blender(vision_path, output_path):
    """Convertit COLMAP vers format Blender/NeRF"""
    os.makedirs(output_path, exist_ok=True)
    images_output = os.path.join(output_path, 'images')
    os.makedirs(images_output, exist_ok=True)

    # Chercher sparse
    sparse_paths = [
        os.path.join(vision_path, "sparse", "0"),
        os.path.join(vision_path, "sparse_text"),
        os.path.join(vision_path, "sparse"),
    ]

    sparse_path = None
    for path in sparse_paths:
        if os.path.exists(path):
            sparse_path = path
            print(f"✅ Sparse trouvé: {path}")
            break

    if not sparse_path:
        print("❌ Dossier sparse introuvable!")
        return False

    # Lire images
    images_bin = os.path.join(sparse_path, 'images.bin')
    images_txt = os.path.join(sparse_path, 'images.txt')

    colmap_data = None
    if os.path.exists(images_bin):
        print("📖 Lecture images.bin...")
        colmap_data = read_colmap_bin_file(images_bin)

    if not colmap_data and os.path.exists(images_txt):
        print("📖 Lecture images.txt...")
        colmap_data = parse_colmap_txt_file(images_txt)

    if not colmap_data:
        print("❌ Impossible de lire les données COLMAP")
        return False

    print(f"✅ {len(colmap_data)} images chargées")

    # Trouver dossier images
    images_dirs = [
        os.path.join(vision_path, "images"),
        os.path.join(vision_path, "..", "images"),
    ]

    images_dir = None
    for img_dir in images_dirs:
        if os.path.exists(img_dir):
            images_dir = img_dir
            break

    if not images_dir:
        print("❌ Dossier images introuvable")
        return False

    print(f"✅ Images source: {images_dir}")

    # Traiter images
    image_files = [f for f in os.listdir(images_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"🖼️ Traitement de {len(image_files)} images...")

    actual_images = []
    for i, img_info in enumerate(colmap_data):
        img_name = img_info.get('name', f'image_{i:04d}')
        base_name = os.path.splitext(img_name)[0]

        src_path = None
        for img_file in image_files:
            if base_name in img_file:
                src_path = os.path.join(images_dir, img_file)
                break

        if src_path and os.path.exists(src_path):
            dst_path = os.path.join(images_output, f"{base_name}.png")

            try:
                img = cv2.imread(src_path, cv2.IMREAD_UNCHANGED)
                if img is None:
                    continue

                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGBA)
                elif img.shape[2] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)

                h, w = img.shape[:2]
                if max(h, w) > 1024:
                    scale = 1024 / max(h, w)
                    new_w, new_h = int(w * scale), int(h * scale)
                    img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

                cv2.imwrite(dst_path, img)
                actual_images.append((i, img_info, base_name))

                if i % 10 == 0:
                    print(f"   ✅ {i+1}/{len(colmap_data)}")

            except Exception as e:
                print(f"   ⚠️ Erreur {img_name}: {e}")

    print(f"✅ {len(actual_images)} images converties")

    # Créer transforms
    transforms = {"camera_angle_x": 0.691, "frames": []}

    for idx, img_info, base_name in actual_images:
        qw = img_info['qw']
        qx = img_info['qx']
        qy = img_info['qy']
        qz = img_info['qz']

        norm = np.sqrt(qw*qw + qx*qx + qy*qy + qz*qz)
        if norm > 0:
            qw, qx, qy, qz = qw/norm, qx/norm, qy/norm, qz/norm

        R = np.array([
            [1-2*qy*qy-2*qz*qz, 2*qx*qy-2*qz*qw, 2*qx*qz+2*qy*qw],
            [2*qx*qy+2*qz*qw, 1-2*qx*qx-2*qz*qz, 2*qy*qz-2*qx*qw],
            [2*qx*qz-2*qy*qw, 2*qy*qz+2*qx*qw, 1-2*qx*qx-2*qy*qy]
        ])

        transform = np.eye(4)
        transform[:3, :3] = R
        transform[:3, 3] = [img_info['tx'], img_info['ty'], img_info['tz']]

        transforms['frames'].append({
            "file_path": f"./images/{base_name}",
            "transform_matrix": transform.tolist()
        })

    # Normaliser poses
    if transforms['frames']:
        positions = np.array([np.array(f['transform_matrix'])[:3, 3]
                             for f in transforms['frames']])
        center = np.mean(positions, axis=0)
        max_dist = np.max(np.linalg.norm(positions - center, axis=1))

        if max_dist > 0:
            scale = 1.5 / max_dist
            for frame in transforms['frames']:
                t = np.array(frame['transform_matrix'])
                t[:3, 3] = (t[:3, 3] - center) * scale
                frame['transform_matrix'] = t.tolist()

    # Split aléatoire
    random.seed(42)
    all_indices = list(range(len(transforms['frames'])))
    random.shuffle(all_indices)

    n_total = len(all_indices)
    n_test = int(n_total * 0.10)
    n_val = int(n_total * 0.10)

    test_indices = set(all_indices[:n_test])
    val_indices = set(all_indices[n_test:n_test + n_val])
    train_indices = set(all_indices[n_test + n_val:])

    train_frames = [transforms['frames'][i] for i in sorted(train_indices)]
    val_frames = [transforms['frames'][i] for i in sorted(val_indices)]
    test_frames = [transforms['frames'][i] for i in sorted(test_indices)]

    # Sauvegarder
    with open(os.path.join(output_path, 'transforms_train.json'), 'w') as f:
        json.dump({"camera_angle_x": transforms["camera_angle_x"], "frames": train_frames}, f, indent=2)

    with open(os.path.join(output_path, 'transforms_val.json'), 'w') as f:
        json.dump({"camera_angle_x": transforms["camera_angle_x"], "frames": val_frames}, f, indent=2)

    with open(os.path.join(output_path, 'transforms_test.json'), 'w') as f:
        json.dump({"camera_angle_x": transforms["camera_angle_x"], "frames": test_frames}, f, indent=2)

    print(f"✅ Dataset créé: {len(train_frames)} train, {len(val_frames)} val, {len(test_frames)} test")
    return True

# Conversion
scene_path = os.path.join(DATASET_ROOT, SCENE_NAME)
if not RESUME_MODE or not os.path.exists(scene_path):
    if os.path.exists(scene_path):
        shutil.rmtree(scene_path)

    success = convert_colmap_to_blender(VISION_PATH, scene_path)
    if not success:
        print("❌ Conversion échouée")
        sys.exit(1)
else:
    print("♻️ Dataset existant réutilisé")

# ==========================================
# 7. ENTRAÎNEMENT
# ==========================================
print("\n[6/7] 🚀 ENTRAÎNEMENT")
print("="*70)

MAX_ITER = 80000

print(f"🎯 Configuration:")
print(f"   Mode       : {'REPRISE' if RESUME_MODE else 'NOUVEAU'}")
print(f"   Groupe     : {GROUP_NAME}")
print(f"   Itérations : {MAX_ITER}")
print(f"   Dataset    : {scene_path}")
print(f"   Checkpoints: {DRIVE_RESULTS}/{GROUP_NAME}")
print("="*70)

cmd = [
    'python', '-u', 'train.py',
    '--yaml=barf_blender',
    '--model=barf',
    f'--group={GROUP_NAME}',
    '--name=training',
    f'--data.scene={SCENE_NAME}',
    f'--data.root={DATASET_ROOT}',
    f'--max_iter={MAX_ITER}',
    '--visdom.port=-1',
    '--freq.ckpt=2000',
    '--freq.val=2000',
    '--freq.vis=4000',
]

if RESUME_MODE:
    cmd.extend(['--resume', '--load=latest'])

print(f"\n▶️ {' '.join(cmd)}\n")
print("="*70)

try:
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    for line in process.stdout:
        print(line, end='', flush=True)

    process.wait()

    print("\n" + "="*70)
    if process.returncode == 0:
        print("✅ ENTRAÎNEMENT TERMINÉ")
    else:
        print(f"⚠️ Code de sortie: {process.returncode}")
    print("="*70)

except KeyboardInterrupt:
    print("\n" + "="*70)
    print("⚠️ INTERRUPTION UTILISATEUR")
    print("="*70)
    print(f"💾 Checkpoints sauvegardés dans:")
    print(f"   {DRIVE_RESULTS}/{GROUP_NAME}")
    print(f"\n🔄 Relancez le script pour reprendre!")
    if 'process' in locals():
        process.terminate()
        process.wait()

except Exception as e:
    print(f"\n❌ ERREUR: {e}")
    import traceback
    traceback.print_exc()

# ==========================================
# RÉSUMÉ FINAL
# ==========================================
print("\n" + "="*70)
print("📊 RÉSUMÉ")
print("="*70)
print(f"✅ BARF installé: {BARF_PATH}")
print(f"✅ Dataset: {scene_path}")
print(f"✅ Groupe: {GROUP_NAME}")
print(f"💾 Checkpoints Drive: {DRIVE_RESULTS}/{GROUP_NAME}/training/")
print(f"📈 TensorBoard logs: {DRIVE_RESULTS}/{GROUP_NAME}/training/tensorboard/")
print("="*70)
print("\n💡 Vérifiez vos checkpoints:")
print(f"   !ls -lh {DRIVE_RESULTS}/{GROUP_NAME}/training/*.ckpt")
print("\n💡 Visualisez TensorBoard (optionnel):")
print(f"   %load_ext tensorboard")
print(f"   %tensorboard --logdir {DRIVE_RESULTS}/{GROUP_NAME}/training/tensorboard")
print("="*70)
print("\n🎯 RÉSULTATS ATTENDUS:")
print("   Durée       : ~6 heures")
print("   PSNR final  : 26-28 dB")
print("   Qualité     : Excellente (8.5/10)")
print("   Checkpoints : ~40 fichiers")
print("   Espace Drive: ~800 MB")
print("="*70)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
training:  57%|█████▋    | 45999/80000 [3:46:27<2:34:48,  3.66it/s, it=46000, loss=0.001]

validating:   0%|          | 0/4 [00:00<?, ?it/s]

validating:   0%|          | 0/4 [00:10<?, ?it/s, loss=0.009]

validating:  25%|██▌       | 1/4 [00:10<00:31, 10.55s/it, loss=0.009]

validating:  25%|██▌       | 1/4 [00:20<00:31, 10.55s/it, loss=0.006]

validating:  50%|█████     | 2/4 [00:20<00:20, 10.17s/it, loss=0.006]

validating:  50%|█████     | 2/4 [00:30<00:20, 10.17s/it, loss=0.010]

validating:  75%|███████▌  | 3/4 [00:30<00:09, 10.00s/it, loss=0.010]

validating:  75%|███████▌  | 3/4 [00:40<00:09, 10.00s/it, loss=0.010]

validating: 100%|██████████| 4/4 [00:40<00:00,  9.93s/it, loss=0.010]

                                                                     [val] loss:8.659e-03
checkpoint saved: (vision_20251207_2007) training, epoch None (iteration 46000)

training:  59%|█████▉    | 47322/80000 [3:53:09<2

In [ ]:
"""
==========================================
BARF VISION - VERSION FINALE CORRIGÉE
Reprise depuis checkpoint 46000 → 80000
Configuration avec path corrigé: training/model/
==========================================
"""

import os
import json
import shutil
import subprocess
import sys
import random
import numpy as np
import cv2
import struct
from datetime import datetime
from google.colab import drive

print("="*70)
print("🔄 BARF VISION - REPRISE DEPUIS CHECKPOINT 46000")
print("="*70)

# ==========================================
# 1. MONTAGE DRIVE & CONFIGURATION
# ==========================================
print("\n[1/7] 📁 CONFIGURATION")
print("-"*70)

drive.mount('/content/drive', force_remount=True)

# Chemins
VISION_PATH = '/content/drive/MyDrive/vision'
BARF_PATH = '/content/barf'
DATASET_ROOT = '/content/drive/MyDrive/vision/images'
SCENE_NAME = 'vision_scene'

DRIVE_ROOT = '/content/drive/MyDrive/barf_outputs'
DRIVE_RESULTS = os.path.join(DRIVE_ROOT, 'results')

os.makedirs(DRIVE_RESULTS, exist_ok=True)

print(f"✅ Chemins configurés")
print(f"   Vision  : {VISION_PATH}")
print(f"   Dataset : {DATASET_ROOT}/{SCENE_NAME}")
print(f"   Résultats: {DRIVE_RESULTS}")

# ==========================================
# 2. INSTALLATION BARF
# ==========================================
print("\n[2/7] 🔧 INSTALLATION BARF")
print("-"*70)

if not os.path.exists(BARF_PATH):
    print("📥 Clonage du repository BARF...")
    subprocess.run([
        'git', 'clone',
        'https://github.com/chenhsuanlin/bundle-adjusting-NeRF.git',
        BARF_PATH
    ], check=True, capture_output=True)

    os.chdir(BARF_PATH)

    print("📦 Installation des dépendances...")

    # PyTorch
    try:
        import torch
        print(f"   ✅ PyTorch {torch.__version__} déjà installé")
    except ImportError:
        subprocess.run([
            'pip', 'install', '-q', 'torch', 'torchvision',
            '--index-url', 'https://download.pytorch.org/whl/cu118'
        ], check=True)
        print("   ✅ PyTorch installé")

    # Requirements BARF
    if os.path.exists('requirements.txt'):
        subprocess.run(['pip', 'install', '-q', '-r', 'requirements.txt'], check=True)
        print("   ✅ Requirements installés")

    # Visdom (EXPLICITE)
    print("📦 Installation visdom...")
    subprocess.run(['pip', 'install', '-q', 'visdom'], check=True)
    print("   ✅ Visdom installé")

    # Submodules
    subprocess.run(['git', 'submodule', 'update', '--init', '--recursive'],
                   check=True, capture_output=True)
    print("   ✅ Submodules initialisés")

    print("✅ BARF installé")
else:
    os.chdir(BARF_PATH)
    print("✅ BARF déjà présent")

    # S'assurer que visdom est installé
    try:
        import visdom
        print("   ✅ Visdom déjà installé")
    except ImportError:
        print("📦 Installation visdom...")
        subprocess.run(['pip', 'install', '-q', 'visdom'], check=True)
        print("   ✅ Visdom installé")

# Configuration SSIM
print("\n🔧 Configuration module SSIM...")
ssim_path = os.path.join(BARF_PATH, 'external', 'pohsun_ssim')
os.makedirs(ssim_path, exist_ok=True)

ssim_file = os.path.join(ssim_path, 'pytorch_ssim.py')
init_file = os.path.join(ssim_path, '__init__.py')

if not os.path.exists(ssim_file):
    subprocess.run([
        'wget', '-q', '-O', ssim_file,
        'https://raw.githubusercontent.com/Po-Hsun-Su/pytorch-ssim/master/pytorch_ssim/__init__.py'
    ], check=True)
    print("   ✅ pytorch_ssim.py téléchargé")

if not os.path.exists(init_file):
    with open(init_file, 'w') as f:
        f.write('from .pytorch_ssim import *\n')
    print("   ✅ __init__.py créé")

# ==========================================
# 3. PATCH VISDOM + TENSORBOARD COMPLET
# ==========================================
print("\n[3/7] 🔧 DÉSACTIVATION VISDOM + CONFIGURATION TENSORBOARD")
print("-"*70)

# Patch 1: Modifier base.py (VISDOM + TENSORBOARD)
base_py_path = os.path.join(BARF_PATH, 'model', 'base.py')
if os.path.exists(base_py_path):
    with open(base_py_path, 'r') as f:
        content = f.read()

    # Remplacer TOUTE la fonction setup_visualizer
    import re

    new_function = '''def setup_visualizer(self,opt):
        # Visdom complètement désactivé pour Colab
        opt.visdom.enabled = False
        self.vis = None

        # Initialiser TensorBoard obligatoirement
        from torch.utils.tensorboard import SummaryWriter
        import os

        tb_path = os.path.join(opt.output_path, 'tensorboard')
        os.makedirs(tb_path, exist_ok=True)
        self.tb = SummaryWriter(log_dir=tb_path)

        print("✅ Visdom désactivé (mode Colab)")
        print(f"✅ TensorBoard initialisé: {tb_path}")'''

    pattern = r'def setup_visualizer\(self,opt\):.*?(?=\n    def |\nclass |\Z)'
    content_modified = re.sub(pattern, new_function, content, flags=re.DOTALL)

    with open(base_py_path, 'w') as f:
        f.write(content_modified)
    print("   ✅ base.py patché (Visdom + TensorBoard)")

# Patch 2: Modifier util.py pour éviter check_socket_open avec port -1
util_py_path = os.path.join(BARF_PATH, 'util.py')
if os.path.exists(util_py_path):
    with open(util_py_path, 'r') as f:
        content = f.read()

    # Ajouter une vérification au début de check_socket_open
    old_check = 'def check_socket_open(hostname,port):'
    new_check = '''def check_socket_open(hostname,port):
    # Protection pour port invalide (Colab)
    if port < 0 or port > 65535:
        return False'''

    if old_check in content and 'Protection pour port invalide' not in content:
        content = content.replace(old_check, new_check)
        with open(util_py_path, 'w') as f:
            f.write(content)
        print("   ✅ util.py patché")

# Patch 2B: Modifier util_vis.py pour désactiver les appels Visdom
util_vis_path = os.path.join(BARF_PATH, 'util_vis.py')
if os.path.exists(util_vis_path):
    # D'abord, restaurer le fichier original si corrompu
    print("   🔄 Vérification util_vis.py...")

    with open(util_vis_path, 'r') as f:
        content = f.read()

    # Si le fichier est corrompu (syntaxe invalide), le restaurer
    if 'returndef' in content or content.count('if vis is None: return') > 10:
        print("   ⚠️ Fichier corrompu détecté, restauration depuis Git...")
        subprocess.run(
            ['git', 'checkout', 'HEAD', 'util_vis.py'],
            cwd=BARF_PATH,
            capture_output=True
        )
        # Recharger le contenu
        with open(util_vis_path, 'r') as f:
            content = f.read()
        print("   ✅ Fichier restauré")

    # Maintenant appliquer le patch proprement
    lines = content.split('\n')
    new_lines = []
    i = 0

    while i < len(lines):
        line = lines[i]
        new_lines.append(line)

        # Si c'est une définition de fonction vis_*
        if line.strip().startswith('def vis_') and 'vis' in line and ':' in line:
            # Vérifier si on a déjà patché cette fonction
            if i + 1 < len(lines):
                next_line = lines[i + 1]
                if 'if vis is None' not in next_line:
                    # Calculer l'indentation
                    indent = len(line) - len(line.lstrip())
                    # Ajouter la protection avec la bonne indentation
                    new_lines.append(' ' * (indent + 4) + 'if vis is None: return')

        i += 1

    # Écrire le fichier modifié
    with open(util_vis_path, 'w') as f:
        f.write('\n'.join(new_lines))

    print("   ✅ util_vis.py patché")

# Patch 3: Modifier la config YAML pour forcer disabled
yaml_path = os.path.join(BARF_PATH, 'options', 'barf_blender.yaml')
if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        lines = f.readlines()

    # Chercher et modifier la section visdom
    new_lines = []
    in_visdom = False

    for line in lines:
        if 'visdom:' in line:
            in_visdom = True
            new_lines.append(line)
            new_lines.append('    enabled: false\n')
            continue

        if in_visdom and line.strip() and not line.startswith(' '):
            in_visdom = False

        if in_visdom and ('port:' in line or 'enabled:' in line):
            continue  # Skip ces lignes, on les a déjà ajoutées

        new_lines.append(line)

    with open(yaml_path, 'w') as f:
        f.writelines(new_lines)
    print("   ✅ YAML patché")

print("✅ Visdom désactivé + TensorBoard configuré (4 niveaux de protection)")

# ==========================================
# 4. FONCTIONS COLMAP
# ==========================================

def read_colmap_bin_file(bin_file):
    """Lit un fichier binaire COLMAP"""
    if not os.path.exists(bin_file):
        return None

    data = []
    with open(bin_file, 'rb') as f:
        try:
            num_elements = struct.unpack('Q', f.read(8))[0]
            print(f"   📊 {num_elements} images à traiter")

            for i in range(num_elements):
                image_id = struct.unpack('I', f.read(4))[0]
                qw, qx, qy, qz = struct.unpack('dddd', f.read(32))
                tx, ty, tz = struct.unpack('ddd', f.read(24))
                camera_id = struct.unpack('I', f.read(4))[0]

                name_bytes = b''
                while True:
                    char = f.read(1)
                    if char == b'\x00' or not char:
                        break
                    name_bytes += char

                name = name_bytes.decode('utf-8', errors='ignore')
                num_points2D = struct.unpack('Q', f.read(8))[0]
                f.seek(num_points2D * 24, 1)

                data.append({
                    'image_id': image_id,
                    'qw': qw, 'qx': qx, 'qy': qy, 'qz': qz,
                    'tx': tx, 'ty': ty, 'tz': tz,
                    'camera_id': camera_id,
                    'name': name
                })

                if (i + 1) % 50 == 0:
                    print(f"   📖 {i + 1}/{num_elements}...")

        except Exception as e:
            print(f"   ⚠️ Erreur lecture: {e}")
            return None

    return data

def parse_colmap_txt_file(txt_file):
    """Parse un fichier texte COLMAP"""
    if not os.path.exists(txt_file):
        return None

    colmap_data = []
    with open(txt_file, 'r') as f:
        lines = [l.strip() for l in f if l.strip() and not l.startswith('#')]

        for i in range(0, len(lines), 2):
            if i < len(lines):
                parts = lines[i].split()
                if len(parts) >= 10:
                    colmap_data.append({
                        'image_id': int(parts[0]),
                        'qw': float(parts[1]),
                        'qx': float(parts[2]),
                        'qy': float(parts[3]),
                        'qz': float(parts[4]),
                        'tx': float(parts[5]),
                        'ty': float(parts[6]),
                        'tz': float(parts[7]),
                        'camera_id': int(parts[8]),
                        'name': parts[9]
                    })

    return colmap_data if colmap_data else None

# ==========================================
# 5. DÉTECTION CHECKPOINT 46000
# ==========================================
print("\n[4/7] 🔍 RECHERCHE CHECKPOINT 46000")
print("-"*70)

def find_checkpoint_46k(output_dir):
    """Trouve le groupe contenant le checkpoint 46000"""
    if not os.path.exists(output_dir):
        return None, None

    groups = [d for d in os.listdir(output_dir)
              if os.path.isdir(os.path.join(output_dir, d))
              and d.startswith('vision_')]

    if not groups:
        return None, None

    # Trier par date (plus récent en premier)
    groups.sort(reverse=True)

    # Chercher le groupe avec checkpoint 46000
    for group in groups:
        # CORRECTION: Les checkpoints sont dans training/model/
        group_path = os.path.join(output_dir, group, 'training', 'model')

        if not os.path.exists(group_path):
            # Essayer aussi sans /model (ancienne structure)
            group_path = os.path.join(output_dir, group, 'training')

        ckpt_46k = os.path.join(group_path, '46000.ckpt')

        if os.path.exists(ckpt_46k):
            print(f"   ✅ Checkpoint 46000 trouvé dans: {group}/training/model/")
            return group, '46000'

        # Vérifier aussi latest.ckpt
        latest_ckpt = os.path.join(group_path, 'latest.ckpt')
        if os.path.exists(latest_ckpt):
            print(f"   📝 latest.ckpt trouvé dans: {group}/training/model/")
            return group, 'latest'

    return groups[0] if groups else None, None

# Lien symbolique
BARF_OUTPUT = os.path.join(BARF_PATH, 'output')
if os.path.exists(BARF_OUTPUT):
    if os.path.islink(BARF_OUTPUT):
        os.unlink(BARF_OUTPUT)
    else:
        shutil.rmtree(BARF_OUTPUT)

os.symlink(DRIVE_RESULTS, BARF_OUTPUT)
print(f"✅ Lien symbolique: {BARF_OUTPUT} → {DRIVE_RESULTS}")

# Chercher checkpoint 46000
RESUME_GROUP, RESUME_CKPT = find_checkpoint_46k(DRIVE_RESULTS)

if not RESUME_GROUP or not RESUME_CKPT:
    print("\n❌ ERREUR: Checkpoint 46000 non trouvé!")
    print("\n💡 Vérifiez manuellement vos checkpoints:")
    print(f"   !ls -lh {DRIVE_RESULTS}/vision_*/training/model/*.ckpt")
    print(f"   OU")
    print(f"   !ls -lh {DRIVE_RESULTS}/vision_*/training/*.ckpt")
    sys.exit(1)

GROUP_NAME = RESUME_GROUP
RESUME_MODE = True

print(f"\n✅ REPRISE CONFIRMÉE")
print(f"   Groupe    : {GROUP_NAME}")
print(f"   Checkpoint: {RESUME_CKPT}.ckpt")

# Vérifier que le dataset existe
scene_path = os.path.join(DATASET_ROOT, SCENE_NAME)
if not os.path.exists(scene_path):
    print(f"\n❌ ERREUR: Dataset introuvable à {scene_path}")
    print("\n💡 Le dataset doit exister car vous reprenez l'entraînement.")
    print("   Vérifiez que les fichiers transforms_*.json sont présents.")
    sys.exit(1)

print(f"✅ Dataset trouvé: {scene_path}")

# Lister les checkpoints existants
training_path = os.path.join(DRIVE_RESULTS, GROUP_NAME, 'training', 'model')
if not os.path.exists(training_path):
    # Essayer ancienne structure
    training_path = os.path.join(DRIVE_RESULTS, GROUP_NAME, 'training')

if os.path.exists(training_path):
    ckpts = [f for f in os.listdir(training_path) if f.endswith('.ckpt')]
    print(f"\n📋 Checkpoints existants ({len(ckpts)}):")
    for ckpt in sorted(ckpts)[-5:]:  # Afficher les 5 derniers
        ckpt_path = os.path.join(training_path, ckpt)
        size_mb = os.path.getsize(ckpt_path) / (1024*1024)
        print(f"   • {ckpt} ({size_mb:.1f} MB)")

# ==========================================
# 6. CONVERSION DATASET (SKIP si existe)
# ==========================================
print("\n[5/7] 🔄 VÉRIFICATION DATASET")
print("-"*70)

def convert_colmap_to_blender(vision_path, output_path):
    """Convertit COLMAP vers format Blender/NeRF"""
    os.makedirs(output_path, exist_ok=True)
    images_output = os.path.join(output_path, 'images')
    os.makedirs(images_output, exist_ok=True)

    # Chercher sparse
    sparse_paths = [
        os.path.join(vision_path, "sparse", "0"),
        os.path.join(vision_path, "sparse_text"),
        os.path.join(vision_path, "sparse"),
    ]

    sparse_path = None
    for path in sparse_paths:
        if os.path.exists(path):
            sparse_path = path
            print(f"✅ Sparse trouvé: {path}")
            break

    if not sparse_path:
        print("❌ Dossier sparse introuvable!")
        return False

    # Lire images
    images_bin = os.path.join(sparse_path, 'images.bin')
    images_txt = os.path.join(sparse_path, 'images.txt')

    colmap_data = None
    if os.path.exists(images_bin):
        print("📖 Lecture images.bin...")
        colmap_data = read_colmap_bin_file(images_bin)

    if not colmap_data and os.path.exists(images_txt):
        print("📖 Lecture images.txt...")
        colmap_data = parse_colmap_txt_file(images_txt)

    if not colmap_data:
        print("❌ Impossible de lire les données COLMAP")
        return False

    print(f"✅ {len(colmap_data)} images chargées")

    # Trouver dossier images
    images_dirs = [
        os.path.join(vision_path, "images"),
        os.path.join(vision_path, "..", "images"),
    ]

    images_dir = None
    for img_dir in images_dirs:
        if os.path.exists(img_dir):
            images_dir = img_dir
            break

    if not images_dir:
        print("❌ Dossier images introuvable")
        return False

    print(f"✅ Images source: {images_dir}")

    # Traiter images
    image_files = [f for f in os.listdir(images_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"🖼️ Traitement de {len(image_files)} images...")

    actual_images = []
    for i, img_info in enumerate(colmap_data):
        img_name = img_info.get('name', f'image_{i:04d}')
        base_name = os.path.splitext(img_name)[0]

        src_path = None
        for img_file in image_files:
            if base_name in img_file:
                src_path = os.path.join(images_dir, img_file)
                break

        if src_path and os.path.exists(src_path):
            dst_path = os.path.join(images_output, f"{base_name}.png")

            try:
                img = cv2.imread(src_path, cv2.IMREAD_UNCHANGED)
                if img is None:
                    continue

                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGBA)
                elif img.shape[2] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)

                h, w = img.shape[:2]
                if max(h, w) > 1024:
                    scale = 1024 / max(h, w)
                    new_w, new_h = int(w * scale), int(h * scale)
                    img = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

                cv2.imwrite(dst_path, img)
                actual_images.append((i, img_info, base_name))

                if i % 10 == 0:
                    print(f"   ✅ {i+1}/{len(colmap_data)}")

            except Exception as e:
                print(f"   ⚠️ Erreur {img_name}: {e}")

    print(f"✅ {len(actual_images)} images converties")

    # Créer transforms
    transforms = {"camera_angle_x": 0.691, "frames": []}

    for idx, img_info, base_name in actual_images:
        qw = img_info['qw']
        qx = img_info['qx']
        qy = img_info['qy']
        qz = img_info['qz']

        norm = np.sqrt(qw*qw + qx*qx + qy*qy + qz*qz)
        if norm > 0:
            qw, qx, qy, qz = qw/norm, qx/norm, qy/norm, qz/norm

        R = np.array([
            [1-2*qy*qy-2*qz*qz, 2*qx*qy-2*qz*qw, 2*qx*qz+2*qy*qw],
            [2*qx*qy+2*qz*qw, 1-2*qx*qx-2*qz*qz, 2*qy*qz-2*qx*qw],
            [2*qx*qz-2*qy*qw, 2*qy*qz+2*qx*qw, 1-2*qx*qx-2*qy*qy]
        ])

        transform = np.eye(4)
        transform[:3, :3] = R
        transform[:3, 3] = [img_info['tx'], img_info['ty'], img_info['tz']]

        transforms['frames'].append({
            "file_path": f"./images/{base_name}",
            "transform_matrix": transform.tolist()
        })

    # Normaliser poses
    if transforms['frames']:
        positions = np.array([np.array(f['transform_matrix'])[:3, 3]
                             for f in transforms['frames']])
        center = np.mean(positions, axis=0)
        max_dist = np.max(np.linalg.norm(positions - center, axis=1))

        if max_dist > 0:
            scale = 1.5 / max_dist
            for frame in transforms['frames']:
                t = np.array(frame['transform_matrix'])
                t[:3, 3] = (t[:3, 3] - center) * scale
                frame['transform_matrix'] = t.tolist()

    # Split aléatoire
    random.seed(42)
    all_indices = list(range(len(transforms['frames'])))
    random.shuffle(all_indices)

    n_total = len(all_indices)
    n_test = int(n_total * 0.10)
    n_val = int(n_total * 0.10)

    test_indices = set(all_indices[:n_test])
    val_indices = set(all_indices[n_test:n_test + n_val])
    train_indices = set(all_indices[n_test + n_val:])

    train_frames = [transforms['frames'][i] for i in sorted(train_indices)]
    val_frames = [transforms['frames'][i] for i in sorted(val_indices)]
    test_frames = [transforms['frames'][i] for i in sorted(test_indices)]

    # Sauvegarder
    with open(os.path.join(output_path, 'transforms_train.json'), 'w') as f:
        json.dump({"camera_angle_x": transforms["camera_angle_x"], "frames": train_frames}, f, indent=2)

    with open(os.path.join(output_path, 'transforms_val.json'), 'w') as f:
        json.dump({"camera_angle_x": transforms["camera_angle_x"], "frames": val_frames}, f, indent=2)

    with open(os.path.join(output_path, 'transforms_test.json'), 'w') as f:
        json.dump({"camera_angle_x": transforms["camera_angle_x"], "frames": test_frames}, f, indent=2)

    print(f"✅ Dataset créé: {len(train_frames)} train, {len(val_frames)} val, {len(test_frames)} test")
    return True

# Le dataset doit déjà exister pour une reprise
if not os.path.exists(scene_path):
    print(f"❌ Dataset introuvable, création nécessaire...")
    success = convert_colmap_to_blender(VISION_PATH, scene_path)
    if not success:
        print("❌ Conversion échouée")
        sys.exit(1)
else:
    print("♻️ Dataset existant réutilisé")

# ==========================================
# 7. ENTRAÎNEMENT (REPRISE)
# ==========================================
print("\n[6/7] 🚀 REPRISE ENTRAÎNEMENT")
print("="*70)

MAX_ITER = 80000

print(f"🎯 Configuration:")
print(f"   Mode       : REPRISE depuis checkpoint {RESUME_CKPT}")
print(f"   Groupe     : {GROUP_NAME}")
print(f"   Itérations : 46000 → 80000 (34000 restantes)")
print(f"   Dataset    : {scene_path}")
print(f"   Checkpoints: {DRIVE_RESULTS}/{GROUP_NAME}")
print(f"   Temps estimé: ~2.5 heures")
print("="*70)

cmd = [
    'python', '-u', 'train.py',
    '--yaml=barf_blender',
    '--model=barf',
    f'--group={GROUP_NAME}',
    '--name=training',
    f'--data.scene={SCENE_NAME}',
    f'--data.root={DATASET_ROOT}',
    f'--max_iter={MAX_ITER}',
    '--visdom.port=-1',
    '--freq.ckpt=2000',
    '--freq.val=2000',
    '--freq.vis=4000',
    '--resume',
    f'--load={RESUME_CKPT}',
]

print(f"\n▶️ {' '.join(cmd)}\n")
print("="*70)
print("⏳ Démarrage de l'entraînement depuis itération ~46000...")
print("="*70)

try:
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )

    for line in process.stdout:
        print(line, end='', flush=True)

    process.wait()

    print("\n" + "="*70)
    if process.returncode == 0:
        print("✅ ENTRAÎNEMENT TERMINÉ")
        print(f"🎉 80000 itérations complètes!")
    else:
        print(f"⚠️ Code de sortie: {process.returncode}")
    print("="*70)

except KeyboardInterrupt:
    print("\n" + "="*70)
    print("⚠️ INTERRUPTION UTILISATEUR")
    print("="*70)
    print(f"💾 Checkpoints sauvegardés dans:")
    print(f"   {DRIVE_RESULTS}/{GROUP_NAME}")
    print(f"\n🔄 Relancez le script pour reprendre!")
    if 'process' in locals():
        process.terminate()
        process.wait()

except Exception as e:
    print(f"\n❌ ERREUR: {e}")
    import traceback
    traceback.print_exc()

# ==========================================
# RÉSUMÉ FINAL
# ==========================================
print("\n" + "="*70)
print("📊 RÉSUMÉ")
print("="*70)
print(f"✅ BARF installé: {BARF_PATH}")
print(f"✅ Dataset: {scene_path}")
print(f"✅ Groupe: {GROUP_NAME}")
print(f"💾 Checkpoints Drive: {DRIVE_RESULTS}/{GROUP_NAME}/training/")
print(f"📈 TensorBoard logs: {DRIVE_RESULTS}/{GROUP_NAME}/training/tensorboard/")
print("="*70)
print("\n💡 Vérifiez vos checkpoints:")
print(f"   !ls -lh {DRIVE_RESULTS}/{GROUP_NAME}/training/*.ckpt")
print("\n💡 Visualisez TensorBoard (optionnel):")
print(f"   %load_ext tensorboard")
print(f"   %tensorboard --logdir {DRIVE_RESULTS}/{GROUP_NAME}/training/tensorboard")
print("="*70)
print("\n🎯 RÉSULTATS ATTENDUS:")
print("   Durée       : ~6 heures")
print("   PSNR final  : 26-28 dB")
print("   Qualité     : Excellente (8.5/10)")
print("   Checkpoints : ~40 fichiers")
print("   Espace Drive: ~800 MB")
print("="*70)

Streaming output truncated to the last 5000 lines.
training:  97%|█████████▋| 77999/80000 [2:31:00<08:47,  3.79it/s, it=78000, loss=0.001]

validating:   0%|          | 0/4 [00:00<?, ?it/s]

validating:   0%|          | 0/4 [00:09<?, ?it/s, loss=0.009]

validating:  25%|██▌       | 1/4 [00:09<00:29,  9.78s/it, loss=0.009]

validating:  25%|██▌       | 1/4 [00:19<00:29,  9.78s/it, loss=0.006]

validating:  50%|█████     | 2/4 [00:19<00:18,  9.48s/it, loss=0.006]

validating:  50%|█████     | 2/4 [00:28<00:18,  9.48s/it, loss=0.010]

validating:  75%|███████▌  | 3/4 [00:28<00:09,  9.38s/it, loss=0.010]

validating:  75%|███████▌  | 3/4 [00:37<00:09,  9.38s/it, loss=0.009]

validating: 100%|██████████| 4/4 [00:37<00:00,  9.34s/it, loss=0.009]

                                                                     [val] loss:8.689e-03
checkpoint saved: (vision_20251207_2007) training, epoch None (iteration 78000)

training: 100%|█████████▉| 79999/80000 [2:40:29<00:00,  3.80it/s, it=8e+4, los